In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train

cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# Desafio 1

## 1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

### A) vectorizar documentos

In [22]:
import random

In [23]:
random_indices = random.sample(range(len(newsgroups_train.data)), 5)

In [51]:
random_indices = [4204,3469,3077,8705,226] # Aclaracion lo hard codee porque sin querer corri todo devuelta y no puse random seed

In [25]:
tfidfvect = TfidfVectorizer()

In [26]:
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [27]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [28]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [36]:
target_names = newsgroups_train.target_names

In [52]:
for idx in random_indices:
    print("=" * 80)
    print(f"Documento seleccionado (ID {idx}):")
    print(f"Categoría: {target_names[y_train[idx]]}")
    print("-" * 80)
    print(newsgroups_train.data[idx][:500], "...")
    print("-" * 80)

    # Calcular similitud con el resto del dataset
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares
    most_similar = np.argsort(cossim)[::-1][1:6]

    print("\nDocumentos más similares:")
    for i, sim_idx in enumerate(most_similar):
        print(f"{i+1}. {target_names[y_train[sim_idx]]} (ID {sim_idx})")

    print("\n")

print("=" * 80)
print("Análisis finalizado.")

Documento seleccionado (ID 4204):
Categoría: alt.atheism
--------------------------------------------------------------------------------

The first premise and the conclusion are not properly translated as identity
statements, since the "is" in those statements is the "is" of predication
rather than of identity.  Instead, they should be translated using a
predicate letter.  Using "g" to designate God and "j" to designate Jesus,
and the predicate letter "E" for the property of being eternal, the
first premise is Eg and the conclusion is Ej.
    The second premise appears to contain an "is" of identity, in which
case it can be prop ...
--------------------------------------------------------------------------------

Documentos más similares:
1. alt.atheism (ID 2570)
2. alt.atheism (ID 9429)
3. alt.atheism (ID 491)
4. talk.religion.misc (ID 5255)
5. talk.politics.mideast (ID 1352)


Documento seleccionado (ID 3469):
Categoría: sci.crypt
---------------------------------------------------

#### Analisis entre los documentos seleccionados

In [53]:
for idx in random_indices:
    print("=" * 100)
    print(f"📌 Documento seleccionado (ID {idx}):")
    print(f"Categoría: {target_names[y_train[idx]]}")
    print("-" * 100)
    print(newsgroups_train.data[idx][:500], "...")  # Mostrar los primeros 500 caracteres
    print("-" * 100)

    # Calcular similitud con el resto del dataset
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares (sin incluir el mismo documento)
    most_similar = np.argsort(cossim)[::-1][1:6]

    print("\n📄 Comparación de contenido con los documentos más similares:")
    for i, sim_idx in enumerate(most_similar):
        print(f"\n🔹 Similaridad {i+1}: {target_names[y_train[sim_idx]]} (ID {sim_idx}) - Score: {cossim[sim_idx]:.4f}")
        print(newsgroups_train.data[sim_idx][:500], "...")  # Mostrar los primeros 500 caracteres
        print("-" * 100)

print("\n✅ Análisis finalizado.")

📌 Documento seleccionado (ID 4204):
Categoría: alt.atheism
----------------------------------------------------------------------------------------------------

The first premise and the conclusion are not properly translated as identity
statements, since the "is" in those statements is the "is" of predication
rather than of identity.  Instead, they should be translated using a
predicate letter.  Using "g" to designate God and "j" to designate Jesus,
and the predicate letter "E" for the property of being eternal, the
first premise is Eg and the conclusion is Ej.
    The second premise appears to contain an "is" of identity, in which
case it can be prop ...
----------------------------------------------------------------------------------------------------

📄 Comparación de contenido con los documentos más similares:

🔹 Similaridad 1: alt.atheism (ID 2570) - Score: 0.2161

:> God is eternal.    [A = B]
:> Jesus is God.      [C = A]
:> Therefore, Jesus is eternal.  [C = B]

:> This works

# 📊 Análisis de Similitud y Contenido

## 📌 Documento 1 (ID 4204) - Categoría: alt.atheism
📝 **Resumen:** Discurso sobre lógica y argumentación en el ateísmo, con énfasis en identidad y predicados lógicos.
✅ **Análisis de similitud**:
   - La selección de documentos es muy acertada.
   - Todos los documentos pertenecen a la misma categoría y discuten sobre lógica y ateísmo.
✅ **Análisis de contenido**:
   - Se mantiene la coherencia en los temas de identidad lógica y argumentos filosóficos.
   - **Similitud muy precisa.**

---

## 📌 Documento 2 (ID 3469) - Categoría: sci.crypt
📝 **Resumen:** Debate sobre la seguridad y privacidad del algoritmo Clipper de Clinton en criptografía.
✅ **Análisis de similitud**:
   - Los primeros 3 documentos son altamente relevantes.
   - Los últimos 2 son de otras categorías y podrían no ser los más adecuados.
✅ **Análisis de contenido**:
   - Relación clara con criptografía en los primeros documentos.
   - Posibles errores en la selección de los últimos documentos.
   - **Se recomienda revisar si hay mejores coincidencias.**

---

## 📌 Documento 3 (ID 3077) - Categoría: comp.graphics
📝 **Resumen:** Pregunta sobre la fecha de lanzamiento de Photoshop II y comentarios sobre software gráfico.
✅ **Análisis de similitud**:
   - Mayormente relacionados con informática y software gráfico.
   - Un documento de política en la lista podría ser un error.
✅ **Análisis de contenido**:
   - La mayoría de los documentos hablan sobre software y gráficos.
   - La inclusión de un documento político sugiere que TF-IDF capturó términos comunes en contextos diferentes.
   - **En general, buena selección con una posible excepción.**

---

## 📌 Documento 4 (ID 8705) - Categoría: comp.sys.ibm.pc.hardware
📝 **Resumen:** Problemas de rendimiento con una tarjeta de video en Windows, describiendo lentitud en la interfaz.
✅ **Análisis de similitud**:
   - Los primeros 2 documentos son claramente relevantes (hardware).
   - Los siguientes 3 documentos están más enfocados en administración de ventanas.
✅ **Análisis de contenido**:
   - Relación clara con problemas de hardware y rendimiento en sistemas Windows/Linux.
   - Los documentos de "comp.windows.x" no son de hardware, pero tienen cierta relación con el problema de interfaz.
   - **Buena selección con ligeras desviaciones en los últimos documentos.**

---

## 📌 Documento 5 (ID 226) - Categoría: talk.politics.mideast
📝 **Resumen:** Crítica sobre el Museo del Holocausto en EE.UU. y su impacto político.
✅ **Análisis de similitud**:
   - Los primeros 2 documentos están correctamente alineados con la temática.
   - Documentos 3 y 4 son de categorías no relacionadas (automóviles y criptografía).
✅ **Análisis de contenido**:
   - Los primeros documentos son sobre política y revisionismo histórico, lo cual es adecuado.
   - El documento sobre automóviles no tiene sentido en este contexto.
   - **La similitud basada en TF-IDF puede estar capturando palabras comunes sin comprender el significado.**

---

# 🔍 **Conclusiones Generales**
📌 **Lo que funcionó bien**:
   - En general, los documentos con alta similitud están bien seleccionados.
   - La similitud coseno basada en TF-IDF es efectiva en temas bien definidos.

⚠️ **Posibles problemas detectados**:
   - En algunos casos, documentos de **categorías no relacionadas** fueron seleccionados.
   - TF-IDF puede **capturar palabras comunes en diferentes contextos** sin entender la semántica real.




## Consigna 2

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.



### Modelo baseline

In [40]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [41]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [42]:
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Modelo mejorado

In [43]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV

In [44]:
X_train_text, y_train = newsgroups_train.data, newsgroups_train.target
X_test_text, y_test = newsgroups_test.data, newsgroups_test.target

Lo que voy a hacer para encontrar modelos mejores que el baseline es iterar sobre distintas configuraciones para la vectorizacion y los modelos de prediccion, aplicando grid search.

! Aclaracion esto puede llegar a tardar unos minutos

In [45]:
vectorizer_params = {
    'ngram_range': [(1,1), (1,2)],  # Unigramas y bigramas
    'max_df': [0.8, 1.0],           # Filtrar términos muy frecuentes
    'min_df': [1, 3],               # Filtrar términos poco frecuentes
    'smooth_idf': [True, False]     # Prueba con y sin suavizado
}


# Distintos modelos
models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

best_model = None
best_f1_score = 0
best_params = {}


for ngram_range in vectorizer_params['ngram_range']:
    for max_df in vectorizer_params['max_df']:
        for min_df in vectorizer_params['min_df']:
            for smooth_idf in vectorizer_params['smooth_idf']:
                print(f"🔍 Probando vectorizador con ngram={ngram_range}, max_df={max_df}, min_df={min_df}, smooth_idf={smooth_idf}")

                # Vectorizar los textos
                vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_df=max_df, min_df=min_df, smooth_idf=smooth_idf, stop_words='english')
                X_train = vectorizer.fit_transform(X_train_text)
                X_test = vectorizer.transform(X_test_text)

                # Entrenar y evaluar ambos modelos
                for model_name, model in models.items():
                    print(f"Entrenando {model_name}...")

                    # Grid Search para ajustar el parámetro alpha
                    param_grid = {'alpha': np.linspace(0.1, 1, 10)}  # Valores de alpha entre 0.1 y 1
                    grid_search = GridSearchCV(model, param_grid, scoring='f1_macro', cv=5)
                    grid_search.fit(X_train, y_train)

                    # Predecir en el conjunto de test con el mejor modelo encontrado
                    best_model_instance = grid_search.best_estimator_
                    y_pred = best_model_instance.predict(X_test)
                    f1 = f1_score(y_test, y_pred, average='macro')

                    print(f"📊 {model_name} - F1-macro: {f1:.4f} (mejor alpha={grid_search.best_params_['alpha']})")

                    # Guardar el mejor modelo
                    if f1 > best_f1_score:
                        best_f1_score = f1
                        best_model = best_model_instance
                        best_params = {
                            'model': model_name,
                            'ngram_range': ngram_range,
                            'max_df': max_df,
                            'min_df': min_df,
                            'smooth_idf': smooth_idf,
                            'alpha': grid_search.best_params_['alpha']
                        }

print("\n✅ Mejor configuración encontrada:")
print(f"Modelo: {best_params['model']}")
print(f"ngram_range: {best_params['ngram_range']}")
print(f"max_df: {best_params['max_df']}")
print(f"min_df: {best_params['min_df']}")
print(f"smooth_idf: {best_params['smooth_idf']}")
print(f"alpha: {best_params['alpha']}")
print(f"Mejor F1-macro: {best_f1_score:.4f}")

🔍 Probando vectorizador con ngram=(1, 1), max_df=0.8, min_df=1, smooth_idf=True
Entrenando MultinomialNB...
📊 MultinomialNB - F1-macro: 0.6726 (mejor alpha=0.1)
Entrenando ComplementNB...
📊 ComplementNB - F1-macro: 0.6972 (mejor alpha=0.2)
🔍 Probando vectorizador con ngram=(1, 1), max_df=0.8, min_df=1, smooth_idf=False
Entrenando MultinomialNB...
📊 MultinomialNB - F1-macro: 0.6738 (mejor alpha=0.1)
Entrenando ComplementNB...
📊 ComplementNB - F1-macro: 0.6976 (mejor alpha=0.2)
🔍 Probando vectorizador con ngram=(1, 1), max_df=0.8, min_df=3, smooth_idf=True
Entrenando MultinomialNB...
📊 MultinomialNB - F1-macro: 0.6818 (mejor alpha=0.1)
Entrenando ComplementNB...
📊 ComplementNB - F1-macro: 0.6866 (mejor alpha=0.2)
🔍 Probando vectorizador con ngram=(1, 1), max_df=0.8, min_df=3, smooth_idf=False
Entrenando MultinomialNB...
📊 MultinomialNB - F1-macro: 0.6815 (mejor alpha=0.1)
Entrenando ComplementNB...
📊 ComplementNB - F1-macro: 0.6855 (mejor alpha=0.2)
🔍 Probando vectorizador con ngram=(1, 

el mejor modelo fue el siguiente:
📊 ComplementNB - F1-macro: 0.7098 (mejor alpha=0.1)
🔍 Probando vectorizador con ngram=(1, 2), max_df=0.8, min_df=1, smooth_idf=False
Ya que tuvo el F1 mas alto.

## 3

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Utilizo el mejor modelo del apartado anterior

In [46]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),   # Unigramas y bigramas
    max_df=0.8,           # Filtra términos demasiado frecuentes
    min_df=1,             # Incluye palabras que aparezcan al menos una vez
    smooth_idf=False,     # No suavizar la IDF
    stop_words='english',
    max_features=10000
)
X_train2 = vectorizer.fit_transform(newsgroups_train.data)


In [47]:
terms = vectorizer.get_feature_names_out()

In [48]:
print(terms[1000:1100])  # Muestro las palabras entre 1000-1100

['aix' 'aj' 'ajr' 'ajw' 'ajz' 'ak' 'aka' 'al' 'alabama' 'alan' 'alarm'
 'alas' 'alaska' 'albeit' 'albert' 'alcohol' 'alex' 'alexander' 'algebra'
 'algorithm' 'algorithms' 'ali' 'alias' 'alice' 'alien' 'alignment'
 'alike' 'alive' 'allah' 'alleged' 'allen' 'allergic' 'allies' 'allocated'
 'allocation' 'allocation unit' 'allow' 'allowed' 'allowing' 'allows'
 'alomar' 'alot' 'alpha' 'alt' 'alt atheism' 'alt security' 'alter'
 'altered' 'alternate' 'alternative' 'alternatives' 'altitude'
 'altogether' 'aluminum' 'ama' 'amateur' 'amazed' 'amazing' 'ambassador'
 'ambulance' 'amend' 'amendment' 'amendments' 'america' 'american'
 'american people' 'americans' 'ames' 'ames arc' 'ami' 'amiga'
 'ammunition' 'amounts' 'amour' 'amp' 'amplifier' 'amps' 'amusing'
 'analog' 'analogy' 'analysis' 'anania' 'anas' 'anatolia' 'ancient'
 'anderson' 'andersson' 'andrew' 'andrew cmu' 'andy' 'anecdotal' 'angeles'
 'angels' 'anger' 'angle' 'angles' 'angry' 'animal' 'animals' 'animation']


In [49]:

selected_words = ["alabama", "algebra", "algorithm", "analyst", "angry"]


# Verificar si las palabras seleccionadas están en los términos del vectorizador
valid_words = [word for word in selected_words if word in terms]
word_indices = [np.where(terms == word)[0][0] for word in valid_words]  # Obtener los índices en la matriz



In [50]:
# Calcular similitud coseno entre las palabras seleccionadas y todas las demás
similarities = cosine_similarity(X_train2.T[word_indices], X_train2.T)

# Mostrar las 5 palabras más similares para cada palabra seleccionada
for i, word in enumerate(valid_words):
    print("=" * 80)
    print(f"📌 Palabra seleccionada: **{word}**")
    print("-" * 80)

    # Obtener las 5 palabras más similares (sin contar la misma palabra)
    most_similar_idx = np.argsort(similarities[i])[::-1][1:6]
    for rank, idx in enumerate(most_similar_idx):
        print(f"{rank+1}. {terms[idx]} - Score: {similarities[i][idx]:.4f}")

    print("\n")

print("✅ Análisis completado.")

📌 Palabra seleccionada: **alabama**
--------------------------------------------------------------------------------
1. confirm - Score: 0.2554
2. disappeared - Score: 0.2553
3. aerospace - Score: 0.2538
4. wayne - Score: 0.2278
5. valve - Score: 0.1839


📌 Palabra seleccionada: **algebra**
--------------------------------------------------------------------------------
1. version comments - Score: 0.5714
2. paperback - Score: 0.4339
3. chemistry - Score: 0.3640
4. udel edu - Score: 0.2935
5. good condition - Score: 0.2579


📌 Palabra seleccionada: **algorithm**
--------------------------------------------------------------------------------
1. encryption algorithm - Score: 0.3218
2. secret - Score: 0.2811
3. escrowed - Score: 0.2373
4. secrecy - Score: 0.2227
5. surfaces - Score: 0.2170


📌 Palabra seleccionada: **angry**
--------------------------------------------------------------------------------
1. 708 - Score: 0.1996
2. ak - Score: 0.1943
3. mot com - Score: 0.1702
4. mot - Sco

Esto es una representación basada en frecuencia y no captura semántica. Esto significa que las palabras seleccionadas no son necesariamente las más parecidas en significado, sino aquellas que aparecen con mayor frecuencia en los mismos documentos que la palabra de referencia